In [1]:
from dataclasses import dataclass
from io import BytesIO

import numpy as np
from numpy.typing import NDArray

from hdf5_serialiser import *

In [3]:
@dataclass
class Base:
    name: str


@serialisable
@dataclass
class DataClass(Base):
    data: NDArray[np.float_]


@serialisable
@dataclass
class Small:
    x: int | None
    y: str


@serialisable
@dataclass
class Big:
    a: dict[str, dict[str, int]]
    s: Small
    d: DataClass


s = Small(x=None, y="three")
d = DataClass(data=np.random.rand(2, 3), name="D")
b = Big(a={"k1": {"k11": 123}, "k2": {"k21": 456}}, s=s, d=d)

buff = BytesIO()
b.serialise(buff)
buff.seek(0)
x: Big = Big.deserialise(buff)
x

serialisable.<locals>.NewCls(a={'k1': {'k11': 123}, 'k2': {'k21': 456}}, s=serialisable.<locals>.NewCls(x=None, y='three'), d=serialisable.<locals>.NewCls(name='D', data=array([[0.4106421 , 0.30339523, 0.49463724],
       [0.61516899, 0.99343663, 0.71815992]])))